In [1]:
import os
import sys
# Use local biomni
sys.path.insert(0, "/home/runai-home/Biomni/")

In [2]:
from biomni.agent import A1
from biomni.config import default_config

from biomni.cogzero.prompt.retriever import RETRIEVER_PROMPT
from biomni.cogzero.prompt.proposer import PROPOSER_PROMPT
from biomni.cogzero.prompt.solver import SOLVER_PROMPT

from biomni.cogzero.canonicalizer import compare_outputs
from biomni.cogzero.utils import clear_persistent_namespace, get_all_resources, parse_solution_as_json

In [3]:
# RECOMMENDED: Modify global defaults for consistency

# All agents AND database queries use these defaults
default_config.path = "/mlbio_scratch/panigrah/agents/Biomni/data/"
default_config.source = "Custom"
default_config.base_url = "http://localhost:8000/v1"
llm_dir = "/mlbio_scratch/kodryan/hf_downloads/"
# llm_name = "Qwen2.5-Coder-32B-Instruct"
llm_name = "gpt-oss-120b"
default_config.llm = os.path.join(llm_dir, llm_name) 
default_config.timeout_seconds = 1200

In [4]:
resources = get_all_resources()
data_lake_descriptions = resources["data_lake"]
data_lake_path = os.path.join(default_config.path, "biomni_data", "data_lake")

🎓 Academic mode: Using all datasets (including non-commercial)

🔧 BIOMNI CONFIGURATION
📋 DEFAULT CONFIG (Including Database LLM):
  Path: /mlbio_scratch/panigrah/agents/Biomni/data/
  Timeout Seconds: 1200
  Llm: /mlbio_scratch/kodryan/hf_downloads/gpt-oss-120b
  Temperature: 0.7
  Use Tool Retriever: True
  Commercial Mode: Academic (all datasets)
  Base Url: http://localhost:8000/v1
  Source: Custom

Checking and downloading missing data lake files...


In [5]:
# Custom items in the data lake (basicaly, COSMIC datasets)
custom_items = []
for item in data_lake_descriptions:
    if "Data lake item:" in item["description"]:
        custom_items.append(item)

sc_related_files = [
    "czi_census_datasets_v4.parquet",
    "mousemine_m8_celltype_signature_geneset.parquet",
    "marker_celltype.parquet",
    "msigdb_human_c8_celltype_signature_geneset.parquet",
]
sc_related_items = [item for item in data_lake_descriptions if item["name"]in sc_related_files]

# print("Custom items:", custom_items)
# print("SC-related items:", sc_related_items)

In [6]:
def get_file_size_mb(file_path):
    size_bytes = os.path.getsize(file_path)
    size_mb = size_bytes / (1024 * 1024)  # convert bytes → MB
    return size_mb

for item in data_lake_descriptions:
    file_path = os.path.join(data_lake_path, item["name"])
    item["size"] = get_file_size_mb(file_path)
    
    print(f"{item['name']} size: {item['size']:.2f} MB")

affinity_capture-ms.parquet size: 5.33 MB
affinity_capture-rna.parquet size: 0.04 MB
BindingDB_All_202409.tsv size: 5956.22 MB
broad_repurposing_hub_molecule_with_smiles.parquet size: 1.09 MB
broad_repurposing_hub_phase_moa_target_info.parquet size: 0.17 MB
co-fractionation.parquet size: 1.42 MB
Cosmic_Breakpoints_v101_GRCh38.csv size: 47.19 MB
Cosmic_CancerGeneCensusHallmarksOfCancer_v101_GRCh38.parquet size: 0.30 MB
Cosmic_CancerGeneCensus_v101_GRCh38.parquet size: 0.10 MB
Cosmic_ClassificationPaper_v101_GRCh38.parquet size: 0.41 MB
Cosmic_Classification_v101_GRCh38.parquet size: 0.41 MB
Cosmic_CompleteCNA_v101_GRCh38.tsv.gz size: 206.41 MB
Cosmic_CompleteDifferentialMethylation_v101_GRCh38.tsv.gz size: 210.81 MB
Cosmic_CompleteGeneExpression_v101_GRCh38.tsv.gz size: 2050.58 MB
Cosmic_Fusion_v101_GRCh38.csv size: 18.43 MB
Cosmic_Genes_v101_GRCh38.parquet size: 2.29 MB
Cosmic_GenomeScreensMutant_v101_GRCh38.tsv.gz size: 3467.30 MB
Cosmic_MutantCensus_v101_GRCh38.csv size: 515.10 MB
Co

In [7]:
large_items = [item for item in (sc_related_items + custom_items) if (item["size"] > 10) and (item["size"] < 1000)]
large_items

[{'name': 'Cosmic_Breakpoints_v101_GRCh38.csv',
  'description': 'Data lake item: Cosmic_Breakpoints_v101_GRCh38.csv',
  'size': 47.18643665313721},
 {'name': 'Cosmic_CompleteCNA_v101_GRCh38.tsv.gz',
  'description': 'Data lake item: Cosmic_CompleteCNA_v101_GRCh38.tsv.gz',
  'size': 206.41485500335693},
 {'name': 'Cosmic_CompleteDifferentialMethylation_v101_GRCh38.tsv.gz',
  'description': 'Data lake item: Cosmic_CompleteDifferentialMethylation_v101_GRCh38.tsv.gz',
  'size': 210.81170654296875},
 {'name': 'Cosmic_Fusion_v101_GRCh38.csv',
  'description': 'Data lake item: Cosmic_Fusion_v101_GRCh38.csv',
  'size': 18.429454803466797},
 {'name': 'Cosmic_MutantCensus_v101_GRCh38.csv',
  'description': 'Data lake item: Cosmic_MutantCensus_v101_GRCh38.csv',
  'size': 515.1032209396362}]

In [8]:
import random

# data_path = "/mlbio_scratch/videnovi/BIBA/papers/paper4/data/dataset1.h5ad"
# data_desc = "All cells from human cardiac tissue with myocarditis and healthy controls"

data_path = "/mlbio_scratch/panigrah/agents/data/nature/paper1/cxg_data.h5ad"
data_desc = "scRNA-seq dataset of T cells isolated from the caecum, small‑intestine lamina propria, mesenteric lymph node, and Peyer’s patches"

# data_item = random.choice(large_items)
# data_path = os.path.join(data_lake_path, data_item["name"])
# data_desc = data_item["description"]

data_part_prompt = f"Path: {data_path}\nDescription: {data_desc}\n\n"
data_part_prompt = data_part_prompt.strip()
print(data_part_prompt)
print(f"File size: {get_file_size_mb(data_path):.2f} MB")

Path: /mlbio_scratch/panigrah/agents/data/nature/paper1/cxg_data.h5ad
Description: scRNA-seq dataset of T cells isolated from the caecum, small‑intestine lamina propria, mesenteric lymph node, and Peyer’s patches
File size: 128.23 MB


Single-cell related datasets in the data lake:
* czi_census_datasets_v4.parquet: "Datasets from the Chan Zuckerberg Initiative's Cell Census."
✅ Highly relevant — this is part of the CZI Cell Census project, which is explicitly built on single-cell RNA-seq data integration and annotation across tissues and organisms.

* marker_celltype.parquet: "Cell type marker genes for identification."
✅ Core resource for cell type annotation in single-cell transcriptomics (used to identify clusters).

* mousemine_m8_celltype_signature_geneset.parquet: "Cell type signature gene sets from MouseMine."
✅ Highly relevant — provides mouse cell-type–specific signatures, often used for cross-species or reference-based annotation in single-cell studies.

* msigdb_human_c8_celltype_signature_geneset.parquet: "Cell type signatures from MSigDB."
✅ Relevant — cell-type signature gene sets from the Molecular Signatures Database (MSigDB), useful for enrichment or label transfer in single-cell workflows.

### Retriever

In [9]:
retriever_agent = A1(resource_indices={})

🎓 Academic mode: Using all datasets (including non-commercial)

🔧 BIOMNI CONFIGURATION
📋 DEFAULT CONFIG (Including Database LLM):
  Path: /mlbio_scratch/panigrah/agents/Biomni/data/
  Timeout Seconds: 1200
  Llm: /mlbio_scratch/kodryan/hf_downloads/gpt-oss-120b
  Temperature: 0.7
  Use Tool Retriever: True
  Commercial Mode: Academic (all datasets)
  Base Url: http://localhost:8000/v1
  Source: Custom

Checking and downloading missing data lake files...


In [10]:
retriever_format_dict = {
    "data_path": data_part_prompt,
    "tools": retriever_agent.retriever._format_resources_for_prompt(resources.get("tools", [])),
    "data_lake": retriever_agent.retriever._format_resources_for_prompt(resources.get("data_lake", [])),
    "libraries": retriever_agent.retriever._format_resources_for_prompt(resources.get("libraries", [])),
}
retriever_prompt = RETRIEVER_PROMPT.format(**retriever_format_dict).strip()

In [11]:
retriever_outputs = retriever_agent.go(retriever_prompt)

Using pre-selected retrieval (no LLM)
================================ Human Message =================================

Your task is to select ONLY the resources that are relevant to analyzing the provided single-cell data.

INPUT DATA:
Path: /mlbio_scratch/panigrah/agents/data/nature/paper1/cxg_data.h5ad
Description: scRNA-seq dataset of T cells isolated from the caecum, small‑intestine lamina propria, mesenteric lymph node, and Peyer’s patches

AVAILABLE_RESOURCES
TOOLS:
0. fetch_supplementary_info_from_doi: Fetches supplementary information for a paper given its DOI and saves it to a specified directory.
1. query_arxiv: Query arXiv for papers based on the provided search query.
2. query_scholar: Query Google Scholar for papers based on the provided search query and return the first search result.
3. query_pubmed: Query PubMed for papers based on the provided search query.
4. search_google: Search using Google search and return formatted results.
5. extract_url_content: Extract the t

In [12]:
selected_indices = retriever_agent.retriever._parse_llm_response(retriever_outputs[1])
selected_indices

{'tools': [75, 77, 83],
 'data_lake': [44, 45, 49],
 'libraries': [2, 4, 12, 13, 16, 28, 29, 30, 32, 33, 36]}

In [13]:
selected_resources = retriever_agent.retriever.retrieve_by_indices(resources, selected_indices)
selected_resources

{'tools': [{'description': 'Annotate cell types based on gene markers and transferred labels using LLM. After leiden clustering, annotate clusters using differentially expressed genes and optionally incorporate transferred labels from reference datasets.',
   'name': 'annotate_celltype_scRNA',
   'optional_parameters': [{'default': 'leiden',
     'description': 'Clustering method to use for cell type annotation',
     'name': 'cluster',
     'type': 'str'},
    {'default': 'claude-3-5-sonnet-20241022',
     'description': 'Language model instance for cell type prediction',
     'name': 'llm',
     'type': 'str'},
    {'default': None,
     'description': 'Transferred cell type composition for each cluster',
     'name': 'composition',
     'type': 'pd.DataFrame'}],
   'required_parameters': [{'default': None,
     'description': 'Name of the AnnData file containing scRNA-seq data',
     'name': 'adata_filename',
     'type': 'str'},
    {'default': None,
     'description': 'Directory 

### Proposer

In [14]:
proposer_agent = A1(resource_indices=selected_indices, proposer_mode=True)

🎓 Academic mode: Using all datasets (including non-commercial)

🔧 BIOMNI CONFIGURATION
📋 DEFAULT CONFIG (Including Database LLM):
  Path: /mlbio_scratch/panigrah/agents/Biomni/data/
  Timeout Seconds: 1200
  Llm: /mlbio_scratch/kodryan/hf_downloads/gpt-oss-120b
  Temperature: 0.7
  Use Tool Retriever: True
  Commercial Mode: Academic (all datasets)
  Base Url: http://localhost:8000/v1
  Source: Custom

Checking and downloading missing data lake files...


In [15]:
proposer_prompt = PROPOSER_PROMPT.format(data_path=data_part_prompt).strip()

In [16]:
proposer_outputs = proposer_agent.go(proposer_prompt)

Using pre-selected retrieval (no LLM)
================================ Human Message =================================

You are the TASK PROPOSER. Given single-cell data file(s), you must THOUGHTFULLY INSPECT the data and compose a realistic, non-trivial, data-grounded task suitable for testing a PhD-level bioinformatics candidate.

INPUT DATA:
Path: /mlbio_scratch/panigrah/agents/data/nature/paper1/cxg_data.h5ad
Description: scRNA-seq dataset of T cells isolated from the caecum, small‑intestine lamina propria, mesenteric lymph node, and Peyer’s patches

CONTEXT & CAPABILITIES
- You may use any available tools to load, explore, and compute on the data.
- Work step-by-step; include explanations or reasoning in each step.
- Keep in mind that **the state persists between code executions**: so if in one step you have created variables or imported modules, these will all persist.
- The final task must be solvable from the provided data alone (not external literature), and its answer must be

<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


================================== Ai Message ==================================

**Thinking:**  
I have inspected the dataset and identified the following key points that will inform the task design:

1. **Metadata composition** – The AnnData object contains 10 831 cells from four distinct tissues  
   *mesenteric lymph node* (4 371 cells), *caecum* (3 509), *lamina propria of small intestine* (2 337) and *small intestine Peyer’s patch* (614).  

2. **Cell‑type annotation** – A curated `cell_type` column lists many immune and stromal populations (e.g., plasma cell, dendritic cell, fibroblast, T‑cell subsets).  

3. **Pre‑processing status** – The object already includes PCA and UMAP embeddings, suggesting that normalization, HVG selection, and neighbor‑graph construction have been performed. No explicit QC metrics are stored, but the data are ready for downstream analyses.  

4. **Variability across tissues** – By computing, for each tissue, the proportion of cells belonging to each c

In [17]:
task_dict = parse_solution_as_json(proposer_outputs[1])
task_dict

{'task_description': "Across the four sampled tissues (mesenteric lymph node, caecum, lamina propria of small intestine, and small intestine Peyer's patch), compute for each cell type the proportion of cells belonging to that type within each tissue. Then calculate the coefficient of variation (CV = standard deviation ÷ mean) of these proportions across the tissues (exclude cell types with mean proportion = 0). Which cell type has the highest CV? Answer format: a single lowercase label (shape: label).",
 'ground_truth_answer': 'glial cell'}

### Solver

In [18]:
solver_agent = A1(resource_indices=selected_indices)

🎓 Academic mode: Using all datasets (including non-commercial)

🔧 BIOMNI CONFIGURATION
📋 DEFAULT CONFIG (Including Database LLM):
  Path: /mlbio_scratch/panigrah/agents/Biomni/data/
  Timeout Seconds: 1200
  Llm: /mlbio_scratch/kodryan/hf_downloads/gpt-oss-120b
  Temperature: 0.7
  Use Tool Retriever: True
  Commercial Mode: Academic (all datasets)
  Base Url: http://localhost:8000/v1
  Source: Custom

Checking and downloading missing data lake files...


In [19]:
solver_format_dict = {
    "task_description": task_dict["task_description"],
    "data_path": data_part_prompt,
}
solver_prompt = SOLVER_PROMPT.format(**solver_format_dict).strip()

In [20]:
solver_outputs = solver_agent.go(solver_prompt)

Using pre-selected retrieval (no LLM)
================================ Human Message =================================

Across the four sampled tissues (mesenteric lymph node, caecum, lamina propria of small intestine, and small intestine Peyer's patch), compute for each cell type the proportion of cells belonging to that type within each tissue. Then calculate the coefficient of variation (CV = standard deviation ÷ mean) of these proportions across the tissues (exclude cell types with mean proportion = 0). Which cell type has the highest CV? Answer format: a single lowercase label (shape: label).

INPUT DATA:
Path: /mlbio_scratch/panigrah/agents/data/nature/paper1/cxg_data.h5ad
Description: scRNA-seq dataset of T cells isolated from the caecum, small‑intestine lamina propria, mesenteric lymph node, and Peyer’s patches
================================== Ai Message ==================================

**Plan**

1. [ ] Load the AnnData file (`cxg_data.h5ad`) and inspect its observation co

In [21]:
solver_answer = parse_solution_as_json(solver_outputs[1])
compare_outputs(solver_answer, task_dict["ground_truth_answer"])

Could not parse JSON, returning a raw solution string.


{'equal': False,
 'pred_canon': 'regulatory t cell',
 'gt_canon': 'glial cell',
 'diff': '\n- GT  : glial cell\n+ Pred: regulatory t cell\nΔ mismatch ~40%'}

In [22]:
msgs = proposer_agent._get_messages_for_processing()

DEBUG: Using conversation state with 10 messages


In [23]:
for m in msgs:
    print(m["type"])
    print(m["content"])
    # print(m["original"])
    print(100 * '-')

human
You are the TASK PROPOSER. Given single-cell data file(s), you must THOUGHTFULLY INSPECT the data and compose a realistic, non-trivial, data-grounded task suitable for testing a PhD-level bioinformatics candidate.

INPUT DATA:
Path: /mlbio_scratch/panigrah/agents/data/nature/paper1/cxg_data.h5ad
Description: scRNA-seq dataset of T cells isolated from the caecum, small‑intestine lamina propria, mesenteric lymph node, and Peyer’s patches

CONTEXT & CAPABILITIES
- You may use any available tools to load, explore, and compute on the data.
- Work step-by-step; include explanations or reasoning in each step.
- Keep in mind that **the state persists between code executions**: so if in one step you have created variables or imported modules, these will all persist.
- The final task must be solvable from the provided data alone (not external literature), and its answer must be deterministic.

PROCEDURE (follow step by step and adapt after inspecting results)
**Important:** Do not proceed 

In [24]:
print(retriever_agent.system_prompt)

You are a helpful biomedical assistant assigned with the task of problem-solving.
To achieve this, you will be using an interactive coding environment equipped with a variety of tool functions, data, and softwares to assist you throughout the process.

Given a task, make a plan first. The plan should be a numbered list of steps that you will take to solve the task. Be specific and detailed.
Format your plan as a checklist with empty checkboxes like this:
1. [ ] First step
2. [ ] Second step
3. [ ] Third step

Follow the plan step by step. After completing each step, update the checklist by replacing the empty checkbox with a checkmark:
1. [✓] First step (completed)
2. [ ] Second step
3. [ ] Third step

If a step fails or needs modification, mark it with an X and explain why:
1. [✓] First step (completed)
2. [✗] Second step (failed because...)
3. [ ] Modified second step
4. [ ] Third step

Always show the updated plan after each step so the user can track progress.

At each turn, you sh

In [25]:
from biomni.cogzero.utils import load_data, save_data

In [26]:
mcq = load_data("/mlbio_scratch/panigrah/agents/CognitiaBench/CongitiaBench/mcq.json")
oeq = load_data("/mlbio_scratch/panigrah/agents/CognitiaBench/CongitiaBench/oeq.json")

In [27]:
mcq_datasets = []

for paper, paper_dict in mcq.items():
    print(100 * '-')
    print(paper)
    print(100 * '-')
    for insight, insight_dict in paper_dict.items():
        print(insight)
        print(insight_dict["mcq_question"])
        print()
        mcq_datasets.append(insight_dict["data"])
        has_h5ad = False
        for k, v in insight_dict["data"].items():
            print(k, v)
            has_h5ad = has_h5ad or k.endswith(".h5ad")
        assert has_h5ad
        print()

    print(100 * '=')

----------------------------------------------------------------------------------------------------
J_paper2
----------------------------------------------------------------------------------------------------
insight #1
**Question1:** Based on the dataset analysis, which cell state in neoplastic cells shows a significant increase in proportion after chemotherapy in high-risk neuroblastoma?
A) ADRN (adrenergic)-proliferating
B) ADRN (adrenergic)-dopaminergic
C) ADRN (adrenergic)-baseline
D) Mesenchymal neoplastic cells

**Answer1:**B,D


**Question2:** The data suggest that patients with which genetic mutation show a decrease in mesenchymal cell state frequencies post-therapy, whereas patients without the mutation do not?
A) Mutated ALK
B) Mutated TP53
C) Mutated KRAS
D) None of the above

**Answer2:** A

/mlbio_scratch/videnovi/BIBA/papers/paper2/data/dataset1.h5ad snRNA-seq of all cells in high-risk Neuroblastoma before and after chemotherapy
/mlbio_scratch/videnovi/BIBA/papers/pape

In [28]:
oeq_datasets = []

for paper, paper_dict in oeq.items():
    print(100 * '-')
    print(paper)
    print(100 * '-')
    for insight, insight_dict in paper_dict.items():
        print(insight)
        print(insight_dict["oe_question"])
        print()
        oeq_datasets.append(insight_dict["data"])
        has_h5ad = False
        for k, v in insight_dict["data"].items():
            print(k, v)
            has_h5ad = has_h5ad or k.endswith(".h5ad")
        assert has_h5ad
        print()

    print(100 * '=')

----------------------------------------------------------------------------------------------------
J_paper2
----------------------------------------------------------------------------------------------------
insight #1
**Question1:** What changes can be observed in cell state proportions before and after chemotherapy in high-risk neuroblastoma patients?

**Answer1:** The proportions for ADRN (adrenergic)-baseline and ADRN-proliferating populations decreased after therapy. Conversely, the ADRN-calcium, ADRN-dopaminergic, and Interm-OXPHOS populations exhibited significant increases after therapy. Mesenchymal neoplastic cells (MES) cells made up <10% of neoplastic cells in most samples, but some samples contained a high frequency of MES cells with significant post-therapy changes.

**Question2:** How does therapy influence mesenchymal (MES) neoplastic cell proportions in patients with and without ALK mutations?

**Answer2:** Patients without ALK mutations showed an increase in mesench

In [29]:
len(mcq_datasets), len(oeq_datasets)

(37, 41)

In [30]:
s = "123"
s = s.replace("5", 's')
s

'123'

In [31]:
all_datasets = {}

def _update_all_datasets(ds_dict):
    for data in ds_dict:
        for k, v in data.items():
            k = k.replace("/mlbio_scratch/videnovi/BIBA/papers/", "/mlbio_scratch/videnovi/papers_BIBA/")
            all_datasets.update({k: v})

_update_all_datasets(mcq_datasets)
_update_all_datasets(oeq_datasets)

In [32]:
len(set(all_datasets.keys())), len(all_datasets.keys())

(29, 29)

In [33]:
all_h5ad_datasets = {}

for k, v in all_datasets.items():
    if k.endswith("h5ad"):
        all_h5ad_datasets.update({k: v})
    else:
        print("NOT H5AD!")
    print(k)
    print(v)
    print(f"{get_file_size_mb(k):.2f} Mb")
    print()

/mlbio_scratch/videnovi/papers_BIBA/paper2/data/dataset1.h5ad
snRNA-seq of all cells in high-risk Neuroblastoma before and after chemotherapy
4486.40 Mb

/mlbio_scratch/videnovi/papers_BIBA/paper2/data/dataset2.h5ad
snRNA-seq of neoplastic cells in high-risk Neuroblastoma before and after chemotherapy
2613.46 Mb

NOT H5AD!
/mlbio_scratch/videnovi/papers_BIBA/paper2/data/genes_used_to_calculate_signature_scores_for_ADRN-like_and_MES-like_neoplastic_cell_states.txt
List of genes used to calculate signature scores for ADRN-like and MES_like neoplastic cell states.
0.00 Mb

/mlbio_scratch/videnovi/papers_BIBA/paper2/data/dataset3.h5ad
snRNA-seq of Macrophages in high-risk Neuroblastoma before and after chemotherapy
169.72 Mb

/mlbio_scratch/videnovi/papers_BIBA/paper4/data/dataset1.h5ad
All cells from human cardiac tissue with myocarditis and healthy controls
14496.87 Mb

/mlbio_scratch/videnovi/papers_BIBA/paper4/data/lymph_dataset.h5ad
Lymphocytes from human cardiac tissue with myocardit

In [34]:
load_data("/mlbio_scratch/kodryan/Biomni/data/all_sc_datasets.json")

{'/mlbio_scratch/videnovi/papers_BIBA/paper2/data/dataset1.h5ad': 'snRNA-seq of all cells in high-risk Neuroblastoma before and after chemotherapy',
 '/mlbio_scratch/videnovi/papers_BIBA/paper2/data/dataset2.h5ad': 'snRNA-seq of neoplastic cells in high-risk Neuroblastoma before and after chemotherapy',
 '/mlbio_scratch/videnovi/papers_BIBA/paper2/data/dataset3.h5ad': 'snRNA-seq of Macrophages in high-risk Neuroblastoma before and after chemotherapy',
 '/mlbio_scratch/videnovi/papers_BIBA/paper4/data/dataset1.h5ad': 'All cells from human cardiac tissue with myocarditis and healthy controls',
 '/mlbio_scratch/videnovi/papers_BIBA/paper4/data/lymph_dataset.h5ad': 'Lymphocytes from human cardiac tissue with myocarditis and healthy controls',
 '/mlbio_scratch/videnovi/papers_BIBA/paper4/data/my_dataset.h5ad': 'Myeloid cells from human cardiac tissue with myocarditis and healthy controls',
 '/mlbio_scratch/videnovi/papers_BIBA/paper5/data/dataset2.h5ad': 'snRNA-seq of human visceral adipose